# BoardBrain 🧠 - AI That Reads Rulebooks For You
### Capstone Project for GenAI Intensive Course with Google

## ⚙️ Overview
BoardBrain is your smart assistant for understanding any board game rulebook. Players simply upload the rulebook of a game in PDF format, and BoardBrain transforms it into an intelligent Q&A experience using Gemini 1.5 Pro. From Settlers of Catan to Monopoly, complex rules are no longer a barrier to fun!

## 🚀 Features
- 📄 Upload and parse PDF rulebooks
- 🔍 Automatically chunk and embed rule text
- 🧠 Retrieve the most relevant rule sections using FAISS
- 🤖 Answer natural language questions with Gemini 1.5 Pro

> Example: *"Can I trade brick for wood in Catan?"*

## 🧠 Gen AI Capabilities Demonstrated

| Capability                        | Applied in BoardBrain |
|----------------------------------|----------------------|
| ✅ Document Understanding         | Extracts and chunks rulebook content from PDFs |
| ✅ Embeddings     | Uses Gemini Embedding API  |
| ✅ Vector Search | FAISS for retrieval |
| ✅ Retrieval-Augmented Generation (RAG) | Combines context with Gemini 1.5 Pro to generate answers |

## 💡 Real-World Problem Solved
Many board games have lengthy, complex rulebooks that discourage casual players. BoardBrain makes learning the rules as easy as asking a question — boosting accessibility and fun for all ages.

## 🔧 Tech Stack
- Gemini 1.5 Pro + Gemini Embedding API
- PyPDF2 (PDF parsing)
- FAISS (vector search)
- numpy

### ✅ Step 1: Install dependencies (Kaggle-ready)
Installs the necessary Python libraries:
- google-generativeai: For using the Gemini models via the API.
- faiss-cpu: Facebook's library for efficient similarity search and clustering of dense vectors.
- PyPDF2: Used to read and extract text from PDF rulebooks.

In [ ]:
!pip install -q google-generativeai==0.3.2 faiss-cpu PyPDF2

### ✅ Step 2: Import libraries
Load required libraries

In [ ]:
import os
import faiss
import json
import numpy as np
import PyPDF2
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

### ✅ Step 3: Set up your API key

To run the following cell, your API key must be stored it in a Kaggle secret named GOOGLE_API_KEY.

If you don't already have an API key, you can grab one from AI Studio.

To make the key available through Kaggle secrets, choose Secrets from the Add-ons menu and follow the instructions to add your key or enable it for this notebook.

In [ ]:
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)  # Replace with your API key

### ✅ Step 4: Extract text from uploaded PDF
Reads a PDF rulebook file and extracts all readable text from it.
Converts each page's content into plain text and concatenates them into one string.


Catan base game rulebook has been fed as input here.

In [ ]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        return "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])

pdf_path = "/kaggle/input/catan-rulebook/catan_base_rules.pdf"  # Update with your file path
pdf_text = extract_text_from_pdf(pdf_path)

### ✅ Step 5: Chunk the rulebook
Splits the large block of rulebook text into manageable chunks.
Each chunk contains roughly max_words words.
This helps keep the context size small for embedding and model inference, which improves performance and relevance.

In [ ]:
def chunk_text(text, max_words=300):
    sentences = text.split('. ')
    chunks, chunk = [], []
    count = 0
    for s in sentences:
        chunk.append(s)
        count += len(s.split())
        if count > max_words:
            chunks.append('. '.join(chunk))
            chunk, count = [], 0
    if chunk:
        chunks.append('. '.join(chunk))
    return chunks

chunks = chunk_text(pdf_text)

### ✅ Step 6: Embed chunks using Gemini
- Uses the models/embedding-001 Gemini embedding model to turn each chunk of rulebook text into a high-dimensional vector (embedding).
- These embeddings represent the semantic meaning of each chunk and allow similarity searches later.


In [ ]:
EMBED_MODEL = "models/embedding-001"

def get_embedding(text):
    response = genai.embed_content(
        model=EMBED_MODEL,
        content=text,
        task_type="retrieval_document"
    )
    return np.array(response['embedding'])

embeddings = np.array([get_embedding(chunk) for chunk in chunks])

### ✅ Step 7: Store in FAISS index
- Creates a FAISS index for efficient vector similarity search.
- Stores all rulebook embeddings so that we can later retrieve the most relevant chunks for a user's question.

In [ ]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

### ✅ Step 8: RAG-based Question Answering
Retrieves relevant context:
- Embeds the user's question.
- Finds the top k=3 most similar chunks from the rulebook using FAISS.
Generates an answer:
- Feeds the retrieved rule text + user question into Gemini 1.5 Pro using a helpful assistant prompt.
- Gemini generates a clear answer based on the provided context.

In [ ]:
def query_boardbrain(user_question):
    q_embed = get_embedding(user_question).reshape(1, -1)
    _, I = index.search(q_embed, k=3)
    relevant_context = "\n\n".join([chunks[i] for i in I[0]])

    prompt = f"""You are BoardBrain, a helpful board game rules assistant. 
    Use the context below to answer the user's question clearly and concisely.

    Context:
    {relevant_context}

    User Question:
    {user_question}
    """
    model = genai.GenerativeModel(model_name="models/gemini-1.5-pro")
    response = model.generate_content(prompt)
    return response.text

### ✅ Step 9: Try a sample query
This line demonstrates a real-time user query.

It shows how BoardBrain responds to a natural language question using the uploaded rulebook's knowledge.

In [ ]:
query = "Can I trade wood for brick?"
print('Query:', query)
print('\nAnswer: ', query_boardbrain(query))

In [ ]:
query = "What happens when I roll 7 on dice?"
print('Query:', query)
print('\nAnswer: ', query_boardbrain(query))

In [ ]:
query = "Do my own settlements interrupt my continuous road?"
print('Query:', query)
print('\nAnswer: ', query_boardbrain(query))

## 🧩 Why It’s Useful
- For new players: Ask quick questions without skimming 20 pages.
- For game night hosts: Resolve disputes instantly.
- For educators or facilitators: Clarify rules while teaching games.

## 🌱 Future Ideas
- 🎤 Audio Q&A via Gemini (Audio Understanding)
- 📸 Image upload for game setups (Image Understanding)
- 🎛️ Live UI with Gradio or Streamlit
- 🧩 Agent to simulate game plays using LLM memory

## 🚀 Use Cases
- An adaptation of BrainBoard can be used to read and understand policy documents, user manuals and textbooks.

## 🔗 Citation
@misc{gen-ai-intensive-course-capstone-2025q1,
    author = {Addison Howard and Brenda Flynn and Myles O'Neill and Nate and Polong Lin},
    title = {Gen AI Intensive Course Capstone 2025Q1},
    year = {2025},
    howpublished = {\url{https://kaggle.com/competitions/gen-ai-intensive-course-capstone-2025q1}},
    note = {Kaggle}
}